In [18]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils
from keras.datasets import cifar10
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

In [26]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [11]:
# Set random seed for purposes of reproducibility
seed = 21
numpy.random.seed(seed)

# Prepping the Data

In [4]:
# loading in the data
     
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 30s 0us/step


# normalize the data

In [5]:
# normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# labeling

In [6]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

# Designing the Model

It's important not to have too many pooling layers, as each pooling discards some data. Pooling too often will lead to there being almost nothing for the densely connected layers to learn about when the data reaches them.

In [10]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(y_test.shape[1]))
model.add(Activation('softmax'))

epochs = 25

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_21 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_22 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 64)       

In [12]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=25, 
          batch_size=64)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 51s 1ms/step - loss: 1.5150 - accuracy: 0.4635 - val_loss: 1.2589 - val_accuracy: 0.5472
Epoch 2/25
50000/50000 [==============================] - 33s 657us/step - loss: 1.0221 - accuracy: 0.6383 - val_loss: 0.8548 - val_accuracy: 0.6976
Epoch 3/25
50000/50000 [==============================] - 33s 665us/step - loss: 0.8467 - accuracy: 0.7019 - val_loss: 0.7396 - val_accuracy: 0.7399
Epoch 4/25
50000/50000 [==============================] - 34s 675us/step - loss: 0.7481 - accuracy: 0.7411 - val_loss: 0.7315 - val_accuracy: 0.7410
Epoch 5/25
50000/50000 [==============================] - 34s 675us/step - loss: 0.6957 - accuracy: 0.7569 - val_loss: 0.6883 - val_accuracy: 0.7610
Epoch 6/25
50000/50000 [==============================] - 34s 681us/step - loss: 0.6555 - accuracy: 0.7714 - val_loss: 0.7002 - val_accuracy: 0.7533
Epoch 7/25
50000/50000 [==============================] - 

In [29]:
plt.figure(figsize=(8, 8))
plot_model(model, 
#            to_file='image/model_CNN_LSTM.png', 
           show_shapes=True,
           show_layer_names=True)
plt.show()

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


<Figure size 576x576 with 0 Axes>

In [25]:
plt.style.use('ggplot')
plot_history(history)

NameError: name 'history' is not defined

In [27]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 83.46%
